# Data I/O and file formats

In this lab we will write Python programs that can read crop yield data collected from harvesters and satellite images of the same field into data structures that we can analyse and visualise.

A solid understanding of how to read and write different types of data from and to files is a key skill for data analysis. This week's lab will build these skills and provide an introduction to:

* files, directories, and data storage
* reading and writing files in Python
* tabular, image, and geospatial file formats
* specialist file formats for the web, big data, and cloud computing
* selecting different data formats for specific analysis or storage tasks

## Setup

### Run the labs

You can run the labs locally on your machine or you can use cloud environments provided by Google Colab. **If you're working with Google Colab be aware that your sessions are temporary and you'll need to take care to save, backup, and download your work.**

<a href="https://colab.research.google.com/github/geog3300-agri3003/coursebook/blob/main/docs/notebooks/week-3_1.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Download data

If you need to download the date for this lab, run the following code snippet. 

In [ ]:
import os
import subprocess

if "data_lab-3_1" not in os.listdir(os.getcwd()):
    subprocess.run('wget "https://github.com/geog3300-agri3003/lab-data/raw/main/data_lab-3_1.zip"', shell=True, capture_output=True, text=True)
    subprocess.run('unzip "data_lab-3_1.zip"', shell=True, capture_output=True, text=True)
    if "data_lab-3_1" not in os.listdir(os.getcwd()):
        print("Has a directory called data_lab-3_1 been downloaded and placed in your working directory? If not, try re-executing this code chunk")
    else:
        print("Data download OK")

### Working in Colab

If you're working in Google Colab, you'll need to install the required packages that don't come with the colab environment.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install xarray[complete]
    !pip install rioxarray
    !pip install mapclassify
    !pip install rasterio

## Data I/O

Data analysis tasks involve reading data stored in files on disks, servers in the cloud, or recorded by sensors. Also, we need to save the results of our analysis or datasets we have generated to files. 

There are a range of data types (e.g. string / text, numeric, datetime) and ways of characterising data such as tabular data, images and arrays, and spatial and non-spatial data. This necessitates storing data with different file formats. It's important to be able to read and write data from and to different file formats into and out of Python data structures that we can analyse in our programs. 

There are costs involved in storing and transferring data. These can be time costs associated with the time taken to read data from disk into the computer's memory or transferring data from one computer to another over a network. Or, they can be financial costs associated with storing the data (the cost of hard drives increases with storage capacity and cloud storage providers charge by the byte).

The term big data refers to the increasing volume, variety, and velocity of data. Larger and more diverse datasets are being generated more quickly. To be able to handle big data it is important to select appropriate file formats for efficient storage and reading / writing (or input / output - I/O).

## Import modules

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import plotly.express as px
import plotly.io as pio
import pprint

# setup renderer
if 'google.colab' in str(get_ipython()):
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "jupyterlab"

## A Python program to read a file

Let's start with a simple program to open a CSV file, read some data from it into memory, and then close the connection to the file.

In [ ]:
# path to csv file
elev_csv_path = os.path.join(os.getcwd(), "data_lab-3_1", "week-3-bf66-elevation.csv")
print("path to elevation csv file:", elev_csv_path)

# open the file and read the first 1000 characters
f = open(elev_csv_path, "r", encoding="utf-8")
data = f.read(1000)
f.close()

print("")
print("The first 1000 characters of the csv file area:")
print(data)

## Files and I/O

### Files

A file is data that is stored on a disk. Data in files is stored as a sequence bytes in binary format (values of zero or one).

![](https://github.com/geog3300-agri3003/coursebook/raw/main/docs/img/week-3-file-storage.jpg)

#### Binary

A binary number is represented using only the digits 1 or 0. The binary number system is a base-2 number system (as it has only two symbols). 

The decimal number `0` in binary is `0`:

$0 = (0 \cdot 2^{0})$

The decimal number `1` in binary is `1`:

$1 = (1 \cdot 2^{0})$

The decimal number `2` in binary is `10`:

$2 = (1 \cdot 2^{1}) + (0 \cdot 2^{0})$

The decimal number `3` in binary is `11`:

$3 = (1 \cdot 2^{1}) + (1 \cdot 2^{0})$

The decimal number `4` in binary is `100`:

$4 = (1 \cdot 2^{2}) + (0 \cdot 2^{1}) + (0 \cdot 2^{0})$

You don't need to know the details of the binary number system here, but the key things to take away are:

* numbers are stored in binary using the digits 1 or 0
* larger numbers require more binary digits
* larger numbers, therefore, require more storage space

There are plenty of resources online to learn more about number systems. This is a short article on <a href="https://www.w3schools.com/ai/ai_history_numbers.asp" target="_blank">The History of Numbers</a>.

**Bits**

A single binary digit is a bit. Looking at the pattern above we can store the numbers 0 and 1 using a 1-bit binary number. We can store the numbers 2 and 3 using a 2-bit binary number. We can store the number 4 using a 3-bit binary number. For every extra bit we double the numbers that can be stored in binary. 

* 1 bits = 2 numbers (0 and 1)
* 2 bits = 4 numbers (0 to 3)
* 3 bits = 8 numbers (0 to 7)
* 4 bits = 16 numbers (0 to 15)
* 5 bits = 32 numbers (0 to 31)
* 6 bits = 64 numbers (0 to 63)
* 7 bits = 128 numbers (0 to 128)
* 8 bits = 256 numbers (0 to 255)

**Bytes**

A byte is an 8-bit binary number. With a single byte we can represent 256 different numbers. Computer storage is measured in bytes: 

* 1 Kilobyte (KB) is about 1,000 bytes.
* 1 Megabyte (MB) is about 1,000,000 bytes.
* 1 Gigabyte (GB) is about 1,000,000,000 bytes.

A greyscale image file can store each pixel's colour as an 8-bit number or as a single byte. Black is represented as the number 0 (00000000), white is the number 255 (11111111), and shades of grey are intermediate numbers (00000001 to 11111110). 

If this greyscale image has 100x100 pixels (10,000 pixels), how many bytes of storage does this image require?

$10000 bytes = 10000 pixels \cdot 1 byte$ as each pixel requires 1 byte of storage.

Generally, a common text character such as upper and lower case letters (A-Z, a-z) and symbols (!, @, # etc.) requires a byte of storage. The CSV file we read above stores text characters. We can use the `os.stats()` function to inspect the stats of this file in storage. Let's pass the path to the `week-2-bf66-elevation.csv` file into the `os.stats()` function and see how many bytes are required to store this file.

In [ ]:
elev_csv_path = os.path.join(os.getcwd(), "week-2", "week-2-bf66-elevation.csv")
file_stats = os.stat(elev_csv_path)
print("File Size in Bytes is:",  file_stats.st_size)

## File formats

A file format describes how data is encoded as binary sequences in files (sequences of 1 and 0 digits). The filename's extension indicates the file format used (e.g. .jpg is a JPEG file, .tif is a TIFF or GeoTIFF file, .csv is a CSV file). Some file formats also include a header or magic number inside the file that indicates what the file format is. The header can also include some metadata information about the file. 

![](https://github.com/geog3300-agri3003/coursebook/raw/main/docs/img/week-3-file-formats.jpg)

A Portable Network Graphics (<a href="" target="_blank">PNG</a>) file is a common file format for storing image data. It is identified by a .png ending and consists of sections of bytes on disk arranged as:

1. PNG signature including a magic number for the start of the file and a PNG file identifier.
2. Image header with image metadata such as the size of the image.
3. Image data.
4. Image end to indicate the end of the PNG file on disk.

![](https://github.com/geog3300-agri3003/coursebook/raw/main/docs/img/week-3-png-format.jpg)

Different file formats encode data in different ways, and, thus, have different strengths and weaknesses. Some file formats prioritise efficient storage (compression) of data on disk (small file sizes and quick transfer), other prioritise quick read and writing of data, and others prioritise cross-platform compatibility or interpretation. 

The various file formats for storing vector geospatial data provide a good illustration of how different formats store the same data but in ways that are better suited for different applications and uses. This is discussed <a href="https://macwright.com/2022/05/30/new-formats.html" target="_blank">here</a> and <a href="http://switchfromshapefile.org/" target="_blank">here</a>.

Many of you will have used shapefiles as a format for storing vector geospatial data. A strength of shapefiles is the range of software applications that can read and write data from and to them. However, they have drawbacks which mean they're not suited to some use cases:

* Multifile format which makes data handling harder (you need a .shp, .shx, .prj, and .dbf file).
* File size is limited to 2 GB which is prohibits their use for storing large datasets.
* Attribute names are limited to 10 characters which can preclude using descriptive attribute and column names.
* Each shapefile can only store one type of geometry (point or line or polygon) which prohibits representing complex geographic features in a single file.
* No null value which can introduce complications for handling missing data.
* Data types are limited to 256 characters which precludes storing large numbers / text strings.

It is important to be aware of the characteristics of particular file formats and what their limits or benefits mean for your analysis. A simple example of why this is important: *we've processed several harvester yield datasets from different fields into a single dataset and want to save this dataset to file. If this dataset is larger than 2GB and we try and save to a shapefile there will be data loss.*

## Directories and file systems

Files are organised within a hierarchy of directories and sub-directories (or folders) in a computer system. We're working in a Linux environment so the directory hierachy starts at the root denoted by `/`. Sub-directories are separated by `/`.

A program has a current working directory which is its current location within the directory hierarchy.

We can get the current working directory by calling the `os.getcwd()` function.

In [ ]:
os.getcwd()

We can get a list of files and sub-directories within the current working directory by calling the `os.listdir()` function.

In [ ]:
os.listdir()

A path describes the location of a file within the computer system's directory structure. We can create paths to files using the `os.path.join()` function. We pass in string data representing sub-directories and filenames and the `os.path.join()` function creates a file path. 

Let's get the file path for this notebook file: `week-3_1.ipynb`.

In [ ]:
os.path.join(os.getcwd(), "week-3_1.ipynb")

#### Recap quiz

**In the directory `data_lab-3_1` there is a file named `week-3-bf66-wheat-yield-max-vi_sampled.geojson`. Can you create a file path for this file using `os.path.join()` and print the path on the display?**

In [ ]:
## ADD CODE HERE

<details>
    <summary><b>answer</b></summary>
    
```python
print(os.path.join(os.getcwd(), "data_lab-3-1", "week-3-bf66-wheat-yield-max-vi_sampled.geojson"))
```
</details>

**Is `data_lab-3_1` a directory or a file?**

<details>
    <summary><b>answer</b></summary>
    
<code>data_lab-3_1</code> is a directory which can store files or sub-directories within the file system. You can print out the list of <code>data_lab-3_1</code>'s contents using the <code>os.listdir(os.path.join(os.getcwd(), "data_lab-3_1"))</code> method. 
</details>

## Files in Python

In Python, it is possible to read and write files in text and binary modes.

### Text mode

Text mode involves reading and writing string data from and to the file. 

A text file contains encoded characters. ASCII and Unicode are character sets that define how characters (e.g. 1, 2, 3, 66, A, b, !) are encoded in binary (sequences of 1 and 0 digits) in a file. 

A character set translates characters into numbers and an encoding translates numbers into binary.

#### ASCII

ASCII stands for the American Standard Code for Information Interchange and has encodings for 128 English characters, numbers, and some special characters. ASCII characters are encoded using 7-bits. You can see the full ASCII character set <a href="https://www.w3schools.com/charsets/ref_html_ascii.asp" target="_blank">here</a>.

In ASCII, uppercase G is represented by the number 71, uppercase I is represented by the number 73, and uppcase S is represented by the number 83.

Thus, GIS in ASCII is written as `71 73 83` in its numeric representation and `01000111 01001001 01010011` in its binary representation (i.e. how it would be stored in a file).

#### Unicode

Unicode is a more modern and comprehensive character set of text symbols covering modern and ancient languages. Common encodings of the Unicode character set are UTF-8 and UTF-16. The Unicode character set includes over 1,000,000 characters and aims to be a universal system for representing and storing text in computer systems.

Unicode characters can be encoded in UTF-8 using one to four bytes. More common symbols (e.g. the ASCII character set) are encoded using one byte for efficient storage. 

The Python <a href="https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files" target="_blank">docs</a> suggest UTF-8 is the modern de-facto standard so it is often the default encoding or a good one to choose if you are not sure how your data is encoded.

### Binary mode

Reading files in binary mode does not assume that specific bytes represent human readable characters. When reading files in binary mode, sequences of bytes are read from the file into Python bytes objects in memory. Images such as JPEG files would be read in binary mode. 

To be clear, both text and binary data is stored in binary format on disks. However, when reading in text data the binary data on disk is converted to text characters based on the encoding scheme used and read into string objects. When reading data in binary mode the binary data is read straight into memory as bytes objects. Reading and writing data in text mode will be slower because of the encoding overhead.


### Opening files

The `open()` function opens a connection to a file on disk, or creates a new file if it does not exist, and returns a `file` connection object. 

Typically, `open()` is called by specifying a filename, mode, and encoding as arguments: `open(filename, mode, encoding)`.

* **filename**: the path and filename of the file to be opened.
* **mode**: the mode to open the connection to file in. To open files in text mode use `r` for read only, `w` for write only, `a` for appending data to the file, `r+` for reading and writing. To open files in binary mode use `rb` for read only, `wb` for write only, and `rb+` for reading and writing.
* **the default is to open connections in text mode - be careful if you are opening a connection to a file that is not text data.**
* **encoding**: the encoding of the data in the file.

Let's open up a connection to a CSV file and explore the file object that's returned.

In [ ]:
# open a connection to a csv file
# path to csv file
elev_csv_path = os.path.join(os.getcwd(), "data_lab-3_1", "week-3-bf66-elevation.csv")

# open the file 
f = open(elev_csv_path, "r", encoding="utf-8")

We've opened a connection to a CSV file in read mode `"r"` and with a `utf-8` encoding. This has returned to us `f`, a variable pointing to a `file` object. 

What kind of object is `f` pointing to?

In [ ]:
type(f)

Variable `f` is pointing at a `file` object, or, more specifically, an `_io.TextIOWrapper` type object. The `IO` stands for input / output which is another way of saying reading and writing data. The `Text` refers to the fact that `f` will be able to read data to string Python objects or write data from string Python objects to binary in files (i.e. it's reading in text mode). The conversion from string to binary and binary to string is determined by the encoding (e.g. UTF-8 or ASCII). 

The `file` object `f` is a connection to a file and it provides the methods to read (write) data from (to) this file via the connection.  

### Reading files

A read operation will copy bytes from the file on disk to the computer's memory. The `file` object,`f`, has the `read()` and `readline()` methods. 

The `read(size=n)` method in text mode will read `n` characters from the file. If `n` is omitted or is a negative number the `read()` function will read all of the characters in the file. 

The `readline()` will read until a newline in the text file. Text files have newline characters that denote the end of a line. On Windows the newline character is `\r\n` and on Linux / MacOS it is `\n`. 

Let's test out the `read()` and `readline()` methods.

In [ ]:
# read first 1000 characters from the csv file connected to by f
chars_1000 = f.read(1000)
print(chars_1000)

In [ ]:
# read 10 lines of the csv file 
for i in range(0, 10):
    print(f.readline())

You might notice that the call to `readline()` does not start with the row of column headers. This is because the `file` object `f` keeps a record of a position in the file up to where it has read bytes from. The call to `readline()` will start reading lines of characters from the file where the previous call to `read(1000)` finished. 

We can use the `tell()` method to see a `file` object's current position in a file.

In [ ]:
print(f"current position in the file is {f.tell()}")

The reason that `read()` or `readline()` provide the option to read data in `n` characters at a time or line by line is to help you avoid reading in more data than can fit in your computer's memory. If you had a large file and called `read()` on it, without specifying the number of characters to read, it could fill up your memory.

As we're reading from the file in text mode, the binary data from the file should be converted to Python string objects. We can check this. 

In [ ]:
type(chars_1000)

### Writing files

The `write()` method of the `file` object writes Python objects (e.g. strings) to files. The data will then be stored on disk in a specified format until it is read again by another program. 

When writing data in text mode Python string objects (characters) are encoded (e.g. using ASCII or UTF-8) and stored as bytes on the disk. 

Let's demonstrate a write operation by writing the 1000 characters stored in `chars_1000` to a new file. First, we need to open a connection to the new file in write mode and specify an encoding.

In [ ]:
f2 = open("write_demo.csv", "w", encoding="utf-8")

Now, we can call the `write()` method of the `file` object `f2` and pass `write()` the variable `chars_1000` which points to 1000 characters stored as a string object. If the write is successful, it should return a number telling us how many characters were written (it should be 1000) and you should be able to see the file *write_demo.csv* in your current working directory.

In [ ]:
f2.write(chars_1000)

### Closing files

After you have finished reading or writing data from and to the file, it is important to `close()` the connection to the file. The `file` object's `close()` method does this. 

Once the `close()` method has been called on a `file` object it is no longer possible to read or write data from and to the file. This is important to prevent accidental data loss or corruption. 

We have two open file connections, let's close them.

In [ ]:
f.close()
f2.close()

#### Recap quiz

**Can you open a connection to the file `"week-3-bf66-terra-precip-monthly.csv"`, read and print the first 20 lines, and close the connection the file?**

In [ ]:
## ADD CODE HERE

<details>
    <summary><b>answer</b></summary>
    
```python
f = open(os.path.join(os.getcwd(), "data_lab-3_1", "week-3-bf66-terra-precip-monthly.csv"), "r")
         
for line in range(0, 20):
    print(f.readline())

f.close()
```
</details>

## Context managers 

Context managers - the "correct" way to read and write data from and to files in Python. 

Above we have gone through the process of opening connections to files, reading and writing data, and closing connections to files. However, there are lots of moving parts to this approach as you need to keep track of which connections are open to which files and to ensure you close connections when they are no longer needed. As applications grow and work with more data this can require handling many file connections which adds complexity and increases potential for mistakes / errors. Such mistakes / errors can result in data loss, corrupting files, or reduced security if file connections to private data are leaked. 

There are two "better" ways to read and write data in Python:

1. using context managers.
2. using methods and functions provided by packages (e.g. pandas `read_csv()` function). 

Let's open a file and read data from it using a context manager. **A context manager ensures that connections to files are properly closed without explicitly having to code for it.** 

To create a context, use the `with` statement. 

We've already demonstrated how to open, read, and write data in text mode from a CSV file. Let's use a context manager to demonstrate how to work with files in binary mode. Above we introduced the PNG file format for storing image data. We have a PNG file showing a Google Earth aerial image of the field we're working in. The first few bytes of the PNG file should be the PNG signature including a magic number for the start of the file and an ASCII representation of the letters PNG. 

In [ ]:
png_file_path = os.path.join(os.getcwd(), "data_lab-3_1", "week-3-bf66-low-res.png")

with open(png_file_path, "rb") as f:
    print(f.read(1)) # read and print the first byte
    print(f.read(3)) # read and print the second, third, and fourth bytes

You will note that we opened a connection to the PNG file in `rb` mode. This indicates we are reading data in binary mode. Here, instead of reading in the data a character at a time (as we did when reading data in text mode) we are reading in `n` bytes of data. 

You will also note when printing the binary data that is read from the PNG file there is a `b` in front of the text. This indicates the data being printed is of bytes type. 

Finally, you will notice that we did not need to `close()` the file connection `f` as this is handled for us by working within the context of the `with` statement. 

This is what the data in the PNG file we have just been reading looks like. 

![](https://github.com/geog3300-agri3003/coursebook/raw/main/docs/img/week-3-bf66-low-res.png)

*Imagery (c) 2022 CNES | Airbus, Imagery (c) 2022 | Airbus, Landsat | Copernicus, Maxar Technologies, Map Data (c) 2022*

#### Recap quiz

**Can you read in the file `"week-3-bf66-elevation.csv"` using a context manager?**

In [ ]:
## ADD CODE HERE

<details>
    <summary><b>answer</b></summary>
 
```python
with open(os.path.join(os.getcwd(), "data_lab-3_1", "week-3-bf66-elevation.csv"), "r") as f:
    print(f.read())
```
</details>

## Pandas I/O

Many Python packages provide functions and methods to read and write data that safely open and close connections to files. 

The pandas `read_csv()` function reads CSV data from disk into a pandas `DataFrame` object in a single line of code without us needing to explicitly close a connection to the file.

Let's read some canola yield data collected by a harvester and stored as a CSV file into our program using the pandas `read_csv()` function.

In [ ]:
canola_yield_df = pd.read_csv(os.path.join(os.getcwd(), "data_lab-3_1", "week-3-bf66-canola-yield.csv"))
print(f"The shape of the canola yield DataFrame is {canola_yield_df.shape}")

In [ ]:
canola_yield_df.head()

Pandas provides a range of convenient functions for reading and writing data - you can find a list of them <a href="https://pandas.pydata.org/docs/user_guide/io.html" target="_blank">here</a>.

#### Recap quiz 

**Go to the <a href="https://www.fao.org/faostat/en/#data/QCL" target="_blank">FAOSTAT Data Crop and Livestock page</a> and pick a country, element (e.g. yield, area, production), item (e.g. crop - maize), and selection years. Download the data as a CSV file. Then, using `pd.read_csv()` read the data into a `DataFrame` object referenced by the variable `df_faostat`. Print the `head()` of the `DataFrame`.**

**Hint: you will need to move the CSV file you download from FOASTAT into your working or data directory.**

In [ ]:
%%HTML
<div style="padding:110.6% 0 0 0;position:relative;"><iframe src="https://player.vimeo.com/video/911479339?badge=0&amp;autopause=0&amp;player_id=0&amp;app_id=58479" frameborder="0" allow="autoplay; fullscreen; picture-in-picture" style="position:absolute;top:0;left:0;width:100%;height:100%;" title="faostat-csv"></iframe></div><script src="https://player.vimeo.com/api/player.js"></script>

In [ ]:
## ADD CODE HERE

<details>
    <summary><b>answer</b></summary>

You will need to change the filename to match the file that you have downloaded. 

```python
df_faostat = pd.read_csv(os.path.join(os.getcwd(), "data_lab-3_1", "FAOSTAT_data_en_2-9-2024.csv"))
df_faostat.head()
```
</details>

## Parquet files

So far we have been working with data in CSV format. The CSV format has many strengths for storing tabular data including:

* many software applications provide tools to read and write CSV data.
* the data structure is relatively intuitive with human readable characters encoded in binary, data values (fields) comprise binary representations of characters and are separated by comma symbols (hence the name), and rows (records) are separated by newline symbols. 
* flexibility to choose different encodings of the text data.

However, CSV files require that each data value is stored even if there is lots of repetition. For example, if there is a column that denotes the field name or id, for every row in the table the field name or id value would be repeated. As datasets get large, this can cause CSV files to increase in size which has subsequent storage costs. 

An alternative file format for storing tabular is parquet. Parquet files are optimised for storage. This provides more efficient use of hard drives, cheaper cloud storage costs, and quicker transmission of data over the internet. 

Parquet files have several optimisations for storing tabular data. Whereas CSV files are based around row storage, parquet files are based on column storage. 

![](https://github.com/geog3300-agri3003/coursebook/raw/main/docs/img/week-3-parquet.jpg)

Parquet files can optimise storage of tabular data using **run length encoding** and **dictionary encoding**. 

Run length encoding is useful for reducing storage when there are runs of the same value within a column. For example, in the `canola_yield_df` `DataFrame` the values in the Crop column repeat. Instead of storing every value, we can store the two values: the value that repeats and the number of repetitions in the column (e.g. `(5, 80755)` - instead of storing the number 5 80,755 times as would be the case in a CSV file we can just store two numbers 5 and 80,755). Run length encoding is not suited for CSV files as the data is stored by row, and, often, within a row you'll have data of different types (e.g. string / text, numeric, and dates). This structure doesn't lend itself to encoding runs of the same value - repitition in tabular data generally runs down columns not across rows. 

Dictionary encoding is useful when we need to store large values (e.g. long names or large numbers). Instead of writing the large value repeatedly in the file a smaller value is written and there is a dictionary which acts as a look up table to correspond the small value to the actual large value. This means the large value only needs to be stored once.

For a single field, we have 80,755 records (rows) in the `DataFrame`. If we scaled up this analysis to work with data collected from harvesters across many fields in Western Australia we would quickly accumulate a large volume of data. At some stage we will hit issues with storing and transferring the data and it might be appropriate to switch from CSV to parquet files for data storage. 

In [ ]:
canola_yield_df.to_parquet(os.path.join(os.getcwd(), "data_lab-3_1", "week-3-bf66-canola-yield.parquet"))

Let's compare the size of the CSV file storing the canola yield data and the parquet file we just saved. The parquet file should be much smaller.

In [ ]:
file_stats_csv = os.stat(os.path.join(os.getcwd(), "data_lab-3_1", "week-3-bf66-canola-yield.csv"))
print("CSV file size in bytes is:",  file_stats_csv.st_size)
file_stats_parquet = os.stat(os.path.join(os.getcwd(), "data_lab-3_1", "week-3-bf66-canola-yield.parquet"))
print("Parquet file size in bytes is:",  file_stats_parquet.st_size)